### PGadmin4 interaction

In [57]:
import psycopg2
import pandas as pd

def create_tables():
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname="Project",
        user="postgres",
        password="SJP0195",
        host="localhost",
        port="5432"
    )
    
    # Create a cursor object
    cursor = conn.cursor()

    # Table Creation SQL Queries
    
    create_plat_table = """
    CREATE TABLE IF NOT EXISTS platforms(
    Name VARCHAR(50) PRIMARY KEY,
    Game VARCHAR(50)
    );
    """

    create_pub_table = """
    CREATE TABLE IF NOT EXISTS publishers(
    Name VARCHAR(50) PRIMARY KEY,
    Game VARCHAR(50)
    );
    """
    
    create_team_table = """
    CREATE TABLE IF NOT EXISTS devteams(
    Name VARCHAR(50) PRIMARY KEY,
    Game VARCHAR(50),
    Size INT
    );
    """
    
    create_dev_table = """
    CREATE TABLE IF NOT EXISTS developers(
    Name VARCHAR(50) PRIMARY KEY,
    Experience INT,
    Game VARCHAR(50),
    Team VARCHAR(50) REFERENCES devteams(Name)
    );
    """
    
    create_game_table = """
    CREATE TABLE IF NOT EXISTS games(
    GameID SERIAL PRIMARY KEY,
    Name VARCHAR(50),
    Price INT,
    Genre VARCHAR(50),
    Rating VARCHAR(50),
    Publisher VARCHAR(50) REFERENCES publishers(Name),
    Platform VARCHAR(50) REFERENCES platforms(Name),
    Developer VARCHAR(50) REFERENCES devteams(Name)
    );
    """
    
    try:
        #Execute SQL Statements to create tables.
        cursor.execute(create_pub_table)
        cursor.execute(create_plat_table)
        cursor.execute(create_team_table)
        cursor.execute(create_dev_table)
        cursor.execute(create_game_table)

        #commit changes to database
        conn.commit()
        print("Tables successfully created")
        
    except Exception as e:
        print(f"Error:{e}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()
    
def sampleDataInsertion():
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname="Project",
        user="postgres",
        password="SJP0195",
        host="localhost",
        port="5432"
    )
    
    # Create a cursor object
    cursor = conn.cursor()

    try:
        platData = ''' INSERT INTO platforms (Name, Game)
        VALUES
        ('Nintendo Switch', 'Mario Odyssey'),
        ('Windows 10', 'Deus Ex'),
        ('Playstation 4', 'Bloodborne');
        '''
        pubData = ''' INSERT INTO publishers
        VALUES
        ('Bethesda', 'Skyrim'),
        ('Nintendo', 'Mario Odyseey'),
        ('From Software', 'Elden Ring'),
        ('Bandai Namco', 'Elden Ring');
        '''
        devteamData = ''' INSERT INTO devteams (Name, Game, Size)
        VALUES 
        ('Team Cherry', 'Silksong', 5),
        ('CD Projekt Red', 'Cyberpunk 2077', 80),
        ('Concerned Ape', 'Stardew Valley', 1),
        ('From Software', 'Elden Ring', 140),
        ('Nintendo', 'Mario Odyssey', 400),
        ('Bethesda', 'Skyrim', 200);
        '''
        devData = ''' INSERT INTO developers (Name, Experience, Game, Team)
        VALUES 
        ('Ari Gibson', '20', 'Silksong', 'Team Cherry'),
        ('Eric Barone', '15', 'Stardew Valley', 'Team Cherry');
        '''
        gameData = ''' INSERT INTO games
        VALUES 
        (001, 'Silksong', 30, 'Metroidvania', 'T', 'Nintendo', 'Nintendo Switch', 'Team Cherry' ),
        (057, 'Cyberpunk 2077', 70, 'Open World RPG', '18+', 'Bandai Namco', 'Windows 10', 'CD Projekt Red'),
        (321, 'Stardew Valley', 20, 'farming sim', 'E', 'Nintendo', 'Windows 10', 'Concerned Ape'),
        (116, 'Elden Ring', 60, 'Soulslike', 'T', 'From Software', 'Windows 10', 'From Software'),
        (270, 'Mario Odyssey', 60, 'Open World Platformer', 'E', 'Nintendo', 'Nintendo Switch', 'Nintendo'),
        (009, 'Skyrim', 30, 'Open World RPG', '18+', 'Bethesda', 'Windows 10', 'Bethesda');
        '''
        #Execution
        cursor.execute(pubData)
        cursor.execute(platData)
        cursor.execute(devteamData)
        cursor.execute(devData)
        cursor.execute(gameData)
        #commit changes to database
        conn.commit()
        print("Data inserted")
        
    except Exception as e:
        print(f"Error:{e}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()
        

def SQLtoHTML(category, search, option, sign):
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname="Project",
        user="postgres",
        password="SJP0195",
        host="localhost",
        port="5432"
    )
    
    # Create a cursor object
    cursor = conn.cursor()

    # Get the data
    if search == None or search == '':
        sql = f"SELECT * FROM {category}"
        cursor.execute(sql)
        result = cursor.fetchall()
    elif sign == "None":
        sql = f"SELECT * FROM {category} WHERE {option} = '{search}'"
        cursor.execute(sql)
        result = cursor.fetchall()
    elif sign != "None" and catefory == price or experience or size:
        sql = f"SELECT * FROM {category} WHERE {option} {sign} {search}"
        cursor.execute(sql)
        result = cursor.fetchall()
    

    #convert the data
    df = pd.DataFrame()
    for x in result:
        df2 = pd.DataFrame(list(x)).T
        df = pd.concat([df,df2])

    df.to_html('templates/sql-data.html')
    return 'sql-data.html'


### Flask implementation

In [59]:
from flask import Flask,request,render_template

app = Flask(__name__)

@app.route("/", methods = ['POST', 'GET'])
def hello_world():
    
    if request.method == "POST":
        category = request.form.get("select")
        search = request.form.get("search")
        sign = request.form.get("sign")
        if category == "games":
            option = request.form.get("options1")
        elif category == "developers":
            option = request.form.get("options2")
        elif category == "devteams":
            option = request.form.get("options3")
        elif category == "platforms":
            option = request.form.get("options4")
        elif category == "publishers":
            option = request.form.get("options5")
        result = SQLtoHTML(category,search,option,sign)

        return render_template(result)
    
    return render_template('ProjectHome.html')

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Apr/2025 10:30:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:32:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 10:33:51] "POST / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\sacor\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sacor\AppData\Local\Programs\Python\Python312\Lib\site-packages\fla